In [8]:
# encoding=utf8
import io
import os
import sys
import requests
from collections import OrderedDict 
import math
import random
import numpy as np
import paddle
from paddle.nn import Embedding
import paddle.nn.functional as F

# 数据处理过程：

	下载》》》》》分词》》》》》构造词典、统计词频、词转化为id》》》》》二次采样法优化语料包

In [9]:
# 下载语料用来训练word2vec
def download():
    # 可以从百度云服务器下载一些开源数据集（dataset.bj.bcebos.com）
    corpus_url = "https://dataset.bj.bcebos.com/word2vec/text8.txt"
    # 使用python的requests包下载数据集到本地
    web_request = requests.get(corpus_url)
    corpus = web_request.content
    # 把下载后的文件存储在当前目录的text8.txt文件内
    with open("./text8.txt", "wb") as f:
        f.write(corpus)
    f.close()
download()#该预料包大小约为84M

In [10]:
# 读取text8数据
def load_text8():
    with open("./text8.txt", "r") as f:
        corpus = f.read().strip("\n")
    f.close()

    return corpus

corpus = load_text8()

# 打印前500个字符，简要看一下这个语料的样子
print(corpus[:500])

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philoso


In [11]:
# 对语料进行预处理（分词）
def data_preprocess(corpus):
    # 由于英文单词出现在句首的时候经常要大写，所以我们把所有英文字符都转换为小写，
    # 以便对语料进行归一化处理（Apple vs apple等）
    corpus = corpus.strip().lower()
    corpus = corpus.split(" ")
    return corpus

corpus = data_preprocess(corpus)
print(corpus[:50])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


**key=lambda x: x[1]** 为对前面的对象中的第二维数据（即value）的值进行排序。 key=lambda 变量：变量[维数] 。维数可以按照自己的需要进行设置。

In [12]:
# 构造词典，统计每个词的频率，并根据频率将每个词转换为一个整数id
def build_dict(corpus):
    # 首先统计每个不同词的频率（出现的次数），使用一个词典记录
    word_freq_dict = dict()
    for word in corpus:
        if word not in word_freq_dict:
            word_freq_dict[word] = 0
        word_freq_dict[word] += 1

    # 将这个词典中的词，按照出现次数排序，出现次数越高，排序越靠前
    # 一般来说，出现频率高的高频词往往是：I，the，you这种代词，而出现频率低的词，往往是一些名词，如：nlp
    word_freq_dict = sorted(word_freq_dict.items(), key = lambda x:x[1], reverse = True)
    
    # 构造3个不同的词典，分别存储，
    # 每个词到id的映射关系：word2id_dict
    # 每个id出现的频率：word2id_freq
    # 每个id到词的映射关系：id2word_dict
    word2id_dict = dict()
    word2id_freq = dict()
    id2word_dict = dict()

    # 按照频率，从高到低，开始遍历每个单词，并为这个单词构造一个独一无二的id
    for word, freq in word_freq_dict:
        curr_id = len(word2id_dict)
        word2id_dict[word] = curr_id
        word2id_freq[word2id_dict[word]] = freq
        id2word_dict[curr_id] = word

    return word2id_freq, word2id_dict, id2word_dict

word2id_freq, word2id_dict, id2word_dict = build_dict(corpus)
vocab_size = len(word2id_freq)
print("there are totoally %d different words in the corpus" % vocab_size)

for _, (word, word_id) in zip(range(50), word2id_dict.items()):
    print("word %s, its id %d, its word freq %d" % (word, word_id, word2id_freq[word_id]))

there are totoally 253854 different words in the corpus
word the, its id 0, its word freq 1061396
word of, its id 1, its word freq 593677
word and, its id 2, its word freq 416629
word one, its id 3, its word freq 411764
word in, its id 4, its word freq 372201
word a, its id 5, its word freq 325873
word to, its id 6, its word freq 316376
word zero, its id 7, its word freq 264975
word nine, its id 8, its word freq 250430
word two, its id 9, its word freq 192644
word is, its id 10, its word freq 183153
word as, its id 11, its word freq 131815
word eight, its id 12, its word freq 125285
word for, its id 13, its word freq 118445
word s, its id 14, its word freq 116710
word five, its id 15, its word freq 115789
word three, its id 16, its word freq 114775
word was, its id 17, its word freq 112807
word by, its id 18, its word freq 111831
word that, its id 19, its word freq 109510
word four, its id 20, its word freq 108182
word six, its id 21, its word freq 102145
word seven, its id 22, its wor

从以上词频序列可知，高频词汇主要是冠词、介词、连词、序数词、系动词、代词等，在语言学上，频次越高的词语所携带的信息就越小，其在语言处理中更冗余。比如：高频词“的”，于“美丽的景色”和“美丽景色”两个词组中所带来的差异不大。

为了减少计算量，提高训练效果，使用二次采样法做优化：

**二次采样法**的主要思想是降低高频词在语料中出现的频次。方法是随机将高频的词抛弃，频率越高，被抛弃的概率就越大；频率越低，被抛弃的概率就越小。标点符号或冠词这样的高频词就会被抛弃，从而优化整个词表的词向量训练效果

In [13]:
# 把语料转换为id序列
def convert_corpus_to_id(corpus, word2id_dict):
    # 使用一个循环，将语料中的每个词替换成对应的id，以便于神经网络进行处理
    corpus = [word2id_dict[word] for word in corpus]
    return corpus

corpus = convert_corpus_to_id(corpus, word2id_dict)
print("%d tokens in the corpus" % len(corpus))
print(corpus[:50])

17005207 tokens in the corpus
[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580, 0, 194, 10, 190, 58, 4, 5, 10712, 214, 6, 1324, 104, 454, 19, 58, 2731, 362, 6, 3672, 0]


In [14]:
# 使用二次采样算法（subsampling）处理语料，强化训练效果
def subsampling(corpus, word2id_freq):
    
    # 这个discard函数决定了一个词会不会被替换，这个函数是具有随机性的，每次调用结果不同
    # 如果一个词的频率很大，那么它被遗弃的概率就很大
    def discard(word_id):
        return random.uniform(0, 1) < 1 - math.sqrt(
            1e-4 / word2id_freq[word_id] * len(corpus))

    corpus = [word for word in corpus if not discard(word)]
    return corpus

corpus = subsampling(corpus, word2id_freq)
print("%d tokens in the corpus" % len(corpus))
print(corpus[:50])

8743613 tokens in the corpus
[5233, 3080, 3133, 741, 476, 10571, 27349, 102, 854, 15067, 58112, 854, 3580, 5, 10712, 214, 1324, 2731, 3672, 708, 371, 53, 97, 1423, 2757, 18, 567, 686, 7088, 5233, 1052, 248, 44611, 2877, 792, 5233, 200, 1134, 19, 2621, 8983, 4147, 6437, 4186, 5233, 1137, 344, 1818, 4860, 6753]


注意：在上下文的获取中，窗口大小设置为固定值，这和skip_word_model不一样

# 构造数据生成器

	构造上下文数据》》》构造打包生成器

In [16]:
# 构造数据，准备模型训练
#中心词前两个词是上文，后两个是下文，合起来得到context 
def build_data(corpus, word2id_dict, word2id_freq, max_window_size = 2, negative_sample_num = 4):
    
    # 使用一个list存储处理好的数据
    dataset = []

    # 从左到右，开始枚举每个中心点的位置
    for center_word_idx in range(2,len(corpus)-2):#
        window_size = max_window_size
        # 当前的中心词就是center_word_idx所指向的词
        center_word = corpus[center_word_idx]
        context_word=[corpus[center_word_idx-2],corpus[center_word_idx-1],corpus[center_word_idx+1],corpus[center_word_idx+2]]
        
        dataset.append([context_word,center_word,1])
        
        i=0
        while i < negative_sample_num:
            negative_word_candidate = random.randint(0, vocab_size-1)
            if negative_word_candidate == center_word:
                continue
            else:
                # 把（context，正样本，label=0）的三元组数据放入dataset中，
                # 这里label=0表示这个样本是个负样本
                dataset.append([context_word, negative_word_candidate, 0])
                i += 1

        if center_word_idx % 100000 == 0:
            print(center_word_idx)
    return dataset

corpus_light = corpus[:int(len(corpus)*0.04)]#取小部分预料进行数据构造
dataset = build_data(corpus_light, word2id_dict, word2id_freq)
print(dataset[:5])

100000
200000
300000
[[[5233, 3080, 741, 476], 3133, 1], [[5233, 3080, 741, 476], 29982, 0], [[5233, 3080, 741, 476], 133467, 0], [[5233, 3080, 741, 476], 99004, 0], [[5233, 3080, 741, 476], 186059, 0]]


结果一览

In [17]:
count=0
for data_co in dataset[:5]:
    count+=1
    print("data %s :" %count)
    for i in data_co[0]:
        print("context word is %s" %id2word_dict[i])
    print("target is %s" %id2word_dict[data_co[1]])
    print("label is %d" %data_co[2])

data 1 :
context word is anarchism
context word is originated
context word is working
context word is class
target is abuse
label is 1
data 2 :
context word is anarchism
context word is originated
context word is working
context word is class
target is kennings
label is 0
data 3 :
context word is anarchism
context word is originated
context word is working
context word is class
target is chapleau
label is 0
data 4 :
context word is anarchism
context word is originated
context word is working
context word is class
target is karnac
label is 0
data 5 :
context word is anarchism
context word is originated
context word is working
context word is class
target is givelet
label is 0



	描述的真实短句是：anarchism originated ---abuse--- used working
    负样本有anarchism originated ---acknowledging--- used working等

In [18]:
# 构造mini-batch，准备对模型进行训练
# 我们将不同类型的数据放到不同的tensor里，便于神经网络进行处理
# 并通过numpy的array函数，构造出不同的tensor来，并把这些tensor送入神经网络中进行训练
def build_batch(dataset, batch_size, epoch_num):
    
    # context_word_batch缓存batch_size*context_num个词
    context_word1_batch = []
    context_word2_batch = []
    context_word3_batch = []
    context_word4_batch = []
    # target_word_batch缓存batch_size个目标词（可以是正样本或者负样本）
    target_word_batch = []
    # label_batch缓存了batch_size个0或1的标签，用于模型训练
    label_batch = []
    
    for epoch in range(epoch_num):
        # 每次开启一个新epoch之前，都对数据进行一次随机打乱，提高训练效果
        random.shuffle(dataset)
        
        for data_co in dataset:
            context_words=data_co[0]
            target_word=data_co[1]
            label=data_co[2]

            # 遍历dataset中的每个样本，并将这些数据送到不同的tensor里
            context_word1_batch.append([context_words[0]])
            context_word2_batch.append([context_words[0]])
            context_word3_batch.append([context_words[0]])
            context_word4_batch.append([context_words[0]])

            target_word_batch.append([target_word])
            label_batch.append(label)

            # 当样本积攒到一个batch_size后，我们把数据都返回回来
            # 在这里我们使用numpy的array函数把list封装成tensor
            # 并使用python的迭代器机制，将数据yield出来
            # 使用迭代器的好处是可以节省内存
            if len(context_word4_batch) == batch_size:
                yield np.array(context_word1_batch).astype("int64"), \
                    np.array(context_word2_batch).astype("int64"), \
                    np.array(context_word3_batch).astype("int64"), \
                    np.array(context_word4_batch).astype("int64"), \
                    np.array(target_word_batch).astype("int64"), \
                    np.array(label_batch).astype("float32")
                
                context_word1_batch = []
                context_word2_batch = []
                context_word3_batch = []
                context_word4_batch = []
                target_word_batch = []
                label_batch = []

    """if len(center_word_batch) > 0:
        yield np.array(context_word_batch).astype("int64"), \
            np.array(target_word_batch).astype("int64"), \
            np.array(label_batch).astype("float32")"""

for _, batch in zip(range(10), build_batch(dataset, 128, 3)):
    print(batch)

# 模型构建
![](https://ai-studio-static-online.cdn.bcebos.com/39709d4192014093a1c6c3d45a99ce2829087d1daa854dc3bdfb43cc7a97857f)


如上所示：
在本实验中，四个context_word经过四个embedding，经相加得到隐含层，再从隐含层到输出层

这里和skip_word是不一样的

In [19]:
#定义CBOW训练网络结构
#使用paddlepaddle的2.0.0版本
#一般来说，在使用paddle训练的时候，我们需要通过一个类来定义网络结构，这个类继承了paddle.nn.layer
class CBOW(paddle.nn.Layer):
    def __init__(self, vocab_size, embedding_size, init_scale=0.1):
        # vocab_size定义了CBOW这个模型的词表大小
        # embedding_size定义了词向量的维度是多少
        # init_scale定义了词向量初始化的范围，一般来说，比较小的初始化范围有助于模型训练
        super(CBOW, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        # 使用Embedding函数构造一个词向量参数
        # 这个参数的大小为：[self.vocab_size, self.embedding_size]
        # 数据类型为：float32
        # 这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        self.embedding1 = Embedding( 
            num_embeddings = self.vocab_size,
            embedding_dim = self.embedding_size,
            weight_attr=paddle.ParamAttr(
                initializer=paddle.nn.initializer.Uniform( 
                    low=-init_scale, high=init_scale)))
        self.embedding2 = Embedding( 
            num_embeddings = self.vocab_size,
            embedding_dim = self.embedding_size,
            weight_attr=paddle.ParamAttr(
                initializer=paddle.nn.initializer.Uniform( 
                    low=-init_scale, high=init_scale)))
        self.embedding3 = Embedding( 
            num_embeddings = self.vocab_size,
            embedding_dim = self.embedding_size,
            weight_attr=paddle.ParamAttr(
                initializer=paddle.nn.initializer.Uniform( 
                    low=-init_scale, high=init_scale)))
        self.embedding4 = Embedding( 
            num_embeddings = self.vocab_size,
            embedding_dim = self.embedding_size,
            weight_attr=paddle.ParamAttr(
                initializer=paddle.nn.initializer.Uniform( 
                    low=-init_scale, high=init_scale)))
        
        # 使用Embedding函数构造另外一个词向量参数
        # 这个参数的大小为：[self.vocab_size, self.embedding_size]
        # 这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        self.embedding_out = Embedding(
            num_embeddings = self.vocab_size,
            embedding_dim = self.embedding_size,
            weight_attr=paddle.ParamAttr(
                initializer=paddle.nn.initializer.Uniform(
                    low=-init_scale, high=init_scale)))

    # 定义网络的前向计算逻辑
    # centext_word是四个tensor（mini-batch），表示上下文共四个词
    # target_words是一个tensor（mini-batch），表示目标词
    # label是一个tensor（mini-batch），表示这个词是正样本还是负样本（用0或1表示）
    # 用于在训练中计算这个tensor中对应词的同义词，用于观察模型的训练效果

    def forward(self, centext_word1,centext_word2,centext_word3,centext_word4,target_words, label):
        # 首先，通过self.embedding参数，将mini-batch中的词转换为词向量
        # 这里center_words和eval_words_emb查询的是一个相同的参数
        # 而target_words_emb查询的是另一个参数

        centext_word1=self.embedding1(centext_word1)
        centext_word2=self.embedding2(centext_word2)
        centext_word3=self.embedding3(centext_word3)
        centext_word4=self.embedding4(centext_word4)
        
        context_words_emb=centext_word1+centext_word2+centext_word3+centext_word4
        target_words_emb = self.embedding_out(target_words)

        # 我们通过点乘的方式计算中心词到目标词的输出概率，并通过sigmoid函数估计这个词是正样本还是负样本的概率。
        word_sim = paddle.multiply(context_words_emb, target_words_emb)
        word_sim = paddle.sum(word_sim, axis=-1)
        word_sim = paddle.reshape(word_sim, shape=[-1])
        pred = F.sigmoid(word_sim)

        # 通过估计的输出概率定义损失函数，注意我们使用的是binary_cross_entropy_with_logits函数
        # 将sigmoid计算和cross entropy合并成一步计算可以更好的优化，所以输入的是word_sim，而不是pred
        loss = F.binary_cross_entropy_with_logits(word_sim, label)
        loss = paddle.mean(loss)

        # 返回前向计算的结果，飞桨会通过backward函数自动计算出反向结果。
        return pred, loss

        

In [21]:
# 开始训练，定义一些训练过程中需要使用的超参数
batch_size = 512
epoch_num = 3
embedding_size = 200
step = 0
learning_rate = 0.001

#定义一个使用word-embedding查询同义词的函数
#这个函数query_token是要查询的词，k表示要返回多少个最相似的词，embed是我们学习到的word-embedding参数
#我们通过计算不同词之间的cosine距离，来衡量词和词的相似度
#具体实现如下，x代表要查询词的Embedding，Embedding参数矩阵W代表所有词的Embedding
#两者计算Cos得出所有词对查询词的相似度得分向量，排序取top_k放入indices列表

def get_similar_tokens(query_token, k, embed1,embed2,embed3,embed4):
    embed=embed1+embed2+embed3+embed4
    W = embed.numpy()
    x = W[word2id_dict[query_token]]
    cos = np.dot(W, x) / np.sqrt(np.sum(W * W, axis=1) * np.sum(x * x) + 1e-9)
    flat = cos.flatten()
    indices = np.argpartition(flat, -k)[-k:]
    indices = indices[np.argsort(-flat[indices])]
    for i in indices:
        print('for word %s, the similar word is %s' % (query_token, str(id2word_dict[i])))

# 将模型放到GPU上训练
paddle.set_device('gpu:0')


CBOW_model = CBOW(vocab_size, embedding_size)

# 构造训练这个网络的优化器
adam = paddle.optimizer.Adam(learning_rate=learning_rate, parameters = CBOW_model.parameters())

# 使用build_batch函数，以mini-batch为单位，遍历训练数据，并训练网络
for context_word1,context_word2,context_word3,context_word4, target_words, label in build_batch(
    dataset, batch_size, epoch_num):
    # 使用paddle.to_tensor，将一个numpy的tensor，转换为飞桨可计算的tensor
    context_word1 = paddle.to_tensor(context_word1)
    context_word2 = paddle.to_tensor(context_word2)
    context_word3 = paddle.to_tensor(context_word3)
    context_word4 = paddle.to_tensor(context_word4)

    target_words_var = paddle.to_tensor(target_words)
    label_var = paddle.to_tensor(label)
    
    # 将转换后的tensor送入飞桨中，进行一次前向计算，并得到计算结果
    pred, loss = CBOW_model(
        context_word1,context_word2,context_word3,context_word4, target_words_var, label_var)

    # 程序自动完成反向计算
    loss.backward()
    # 程序根据loss，完成一步对参数的优化更新
    adam.step()
    # 清空模型中的梯度，以便于下一个mini-batch进行更新
    adam.clear_grad()

    # 每经过100个mini-batch，打印一次当前的loss，看看loss是否在稳定下降
    step += 1
    if step % 1000 == 0:
        print("step %d, loss %.3f" % (step, loss.numpy()[0]))

    # 每隔10000步，打印一次模型对以下查询词的相似词，这里我们使用词和词之间的向量点积作为衡量相似度的方法，只打印了5个最相似的词
    if step % 10000 ==0:
        get_similar_tokens('movie', 5, CBOW_model.embedding1.weight,CBOW_model.embedding2.weight,CBOW_model.embedding3.weight,CBOW_model.embedding4.weight)
        get_similar_tokens('one', 5, CBOW_model.embedding1.weight,CBOW_model.embedding2.weight,CBOW_model.embedding3.weight,CBOW_model.embedding4.weight)
        get_similar_tokens('chip', 5, CBOW_model.embedding1.weight,CBOW_model.embedding2.weight,CBOW_model.embedding3.weight,CBOW_model.embedding4.weight)

step 1000, loss 0.694
step 2000, loss 0.618
step 3000, loss 0.476
step 4000, loss 0.154
step 5000, loss 0.147
step 6000, loss 0.105
step 7000, loss 0.022
step 8000, loss 0.018
step 9000, loss 0.018
step 10000, loss 0.013
for word movie, the similar word is movie
for word movie, the similar word is shadowy
for word movie, the similar word is comparison
for word movie, the similar word is kibbutz
for word movie, the similar word is ethereal
for word one, the similar word is one
for word one, the similar word is meets
for word one, the similar word is technological
for word one, the similar word is medecine
for word one, the similar word is harlot
for word chip, the similar word is chip
for word chip, the similar word is permanent
for word chip, the similar word is philosophically
for word chip, the similar word is trail
for word chip, the similar word is parlance


完成任务